# Steps to reproduce this environment to be able to use openmm-torch
Disclaimer: All solutions and fixes were suggestions found in github forums adapted for the present problem.

1. Install pytorch 1.13.0 (preferrable from conda-forge channel), using CUDA 11.7 version.
2. Install openmm from conda as well
3. Try to install openmm-torch with conda
4. Fail and die
5. Rise from the ashes like a phoenix and install openmm-torch from source like a pro (steps to install below) (to confirm if we need to install from source)
6. Install caffee from conda
7. Add the following env variables: `export LD_LIBRARY_PATH=/path/to/caffe2/build/lib:$LD_LIBRARY_PATH` and `export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64:$LD_LIBRARY_PATH`
8. By this point you will have to recompile openmm-torch
9. In my case (Maybe not needed at all) had to uninstall pytorch and install it again.

Note: You need to import torch before importing openmmtorch, otherwise some features won't work

### Steps to install openmm-torch from source

1. Go to `https://pytorch.org/get-started/locally/` and select the combination: Stable, Linux, Libtorch, C++/Java, CUDA 11.7, and copy the command below the description "Download here (cxx11 ABI):". Then change the version from 2.0.1 to 1.13.0. Or alternatively, just copy the following command in the terminal: `wget https://download.pytorch.org/libtorch/cu117/libtorch-shared-with-deps-1.13.0%2Bcu117.zip`
2. unzip the zipped file
3. Make sure the folder where you unzipped libtorch (The thing downloaded above) is accessible from where you're going to build openmm-torch
4. Clone locally the repo https://github.com/openmm/openmm-torch
5. write in the terminal `ccmake openmm-torch`
6. In the interactive terminal, set `PYTORCH_DIR` to point to the directory where you installed the LibTorch.
7. Set `OPENMM_DIR` to point to the directory where OpenMM is installed. This is needed to locate the OpenMM header files and libraries. If you are unsure of what directory this is, the following script will print it out. Set the `CMAKE_INSTALL_PREFIX` to point to the same directory as the `OPENMM_DIR`.
8. Set the `CUDA_TOOLKIT_ROOT_DIR` because in this case we need it. And make sure `NN_BUILD_CUDA_LIB` is selected. In our case we also selected the flags `NN_BUILD_OPENCL_LIB`, `NN_BUILD_PYTHON_WRAPPERS` and the `CUDNN_STATIC`.
9. If it's the first time configuring the build, you may need to press the "configure" `[c]` option twice. After that the "generate" `[g]` option will appear. If the progress percentage reaches 100%


In [ ]:
import torch
from openmmtorch import TorchForce
import os

import yaml
from pathlib import Path

import numpy as np
from openmm.app import *
from openmm import *
from openmm.unit import *
# from force_mapper import ForceMapper

# import mdtraj as md

import training_modules as tm
from sys import stdout

from simutils import ForceReporter, ForceModelConvert

In [ ]:
current_dir = os.getcwd()
fmartip_dir = os.path.dirname(current_dir)
dataset_dir = os.path.dirname(fmartip_dir)
output_file = os.path.join(dataset_dir, "/storage_common/angiod/chignolin_unfolded/chignolin.data.npz")  #fmartip/ff-naive/DatasetsA2A/dataset.A2a.hydrogen.pose0.npz
dataset = dict(np.load(output_file))
# Prepare a simulation system
atomic_numbers = dataset['atom_types']
# output_file = os.path.join(dataset_dir, "sorted.bond.chig.indices.npz")  # sorted.bond.a2a.indices.npz
# bond_indices = np.load(output_file)
# # bond_indices = np.array([[14, 18], [0, 3], [6, 7], [0, 5], [19, 21]])
# dataset['bond_indices'] = np.array(bond_indices['arr_0']).reshape(-1,2)
# config_file = os.path.join(dataset_dir, "config.naive.bonds.yaml")
# conf_bonds: list = yaml.safe_load(Path(config_file).read_text())
# config_file = os.path.join(dataset_dir, "config.naive.angles.yaml")
# conf_angles: list = yaml.safe_load(Path(config_file).read_text())
# config_file = os.path.join(dataset_dir, "config.naive.dihedrals.yaml")
# conf_dihedrals: list = yaml.safe_load(Path(config_file).read_text())

In [ ]:
# torch.cuda.set_device(0)

In [ ]:
dataset.keys()

In [ ]:
dataset['pbc']

In [ ]:
for k, v in dataset.items():
    try:
        print(k, v.shape)
    except:
        print(k)

In [ ]:
dataset['atom_names'][dataset["bead2atom_idcs"][-1]]

In [ ]:
index = 0
# bead mass calculation
bead_mass_dict = []
for atoms in dataset["bead2atom_idcs"]:
    bead_mass = 0
    num_atoms = 0
    for j in atoms[atoms > -1]:
        atom_type = dataset['atom_types'][j]
        if atom_type == 1:
            bead_mass += 1
        elif atom_type == 6:
            bead_mass += 12
        elif atom_type == 7:
            bead_mass += 14
        elif atom_type == 8:
            bead_mass += 16
        elif atom_type == 16:
            bead_mass += 32
        num_atoms += 1
    bead_mass_dict.append(bead_mass)
    # force_matrix[index] = force_matrix[index] / bead_mass * num_atoms
    name_index += 1
    index += 1
dataset['bead_mass'] = bead_mass_dict

In [ ]:
dataset['bead_mass'][-1]

In [ ]:

pdb_file = os.path.join(dataset_dir,  "/home/enere@usi.ch/CGffap/ChigStartingCG.pdb" ) #Chignolin_CG_Unfolded  A2A-CG.pdb ChigStartingCG.pdb#'/home/enere@usi.ch/FMartIP/original_CG_A2A.pdb' "ChignCG_unfolded.pdb" "original_CG_A2A.pdb" "chig_CG/original_CG_a2a_Water.pdb" 
# "/home/enere@usi.ch/FMartIP/chig_CG/original_CG_a2a_4.pdb"
pdb = PDBFile(pdb_file) # OpenMM loader

In [ ]:
# pdb = PDBFile('/home/enere@usi.ch/CGffap/temp/chignolin_unfolded_frames0.pdb') # OpenMM loader

In [ ]:
pdb.getPositions() == pdb.positions

In [ ]:
bead_mass_dict = {}
for idname, mass in zip(dataset['bead_idnames'], dataset['bead_mass']):
    bead_mass_dict[idname] = mass

bead_mass_dict

In [ ]:
pdb_file = os.path.join(dataset_dir, "/home/angiod@usi.ch/CGffap/ChigCG.pdb" ) #'/home/enere@usi.ch/FMartIP/original_CG_A2A.pdb' "ChignCG_unfolded.pdb" "original_CG_A2A.pdb" "chig_CG/original_CG_a2a_Water.pdb" 
# "/home/enere@usi.ch/FMartIP/chig_CG/original_CG_a2a_4.pdb"
pdb = PDBFile(pdb_file) # OpenMM loader

In [ ]:
for index, (atom, bead) in enumerate(zip(pdb.topology.atoms(), np.unique(dataset['bead_idnames']))):
    # print(chr(index + 150))
    i = dataset['bead_types'][np.where(dataset['bead_idnames'] == bead)]
    print(i[0]+100)
    mass = bead_mass_dict[bead]
    print(mass*amu)
    print(bead)
    try:
        atom.element = Element(number = i[0]+50, name = bead, symbol = str(index), mass = mass*amu)
    except:
        atom.element = Element.getByAtomicNumber(i[0]+50)
    index +=1
    print(Element.getByAtomicNumber(i[0]+50))
    print(chr(index))

In [ ]:
bead_charges = []
for bead in dataset['bead_idnames']:
    if bead == 'GLU_SC1' or bead == 'ASP_SC1' :
        bead_charges.append(-1)
    elif bead == 'ARG_SC2' or bead == 'HIS_SC3' or bead == 'LYS_SC2':
        bead_charges.append(+1)
    else:
        bead_charges.append(0)
print(bead_charges)
dataset['bead_charges'] = np.asanyarray(bead_charges).reshape(-1,1)
np.matmul(dataset['bead_charges'], dataset['bead_charges'].T)

In [ ]:
for atom in pdb.topology.atoms():
    atom
    break

In [ ]:
atom.element.mass

In [ ]:
system = System()

for atom in pdb.topology.atoms():
    # print(atom)
    # print(dataset['bead_mass'][atom.index])
    system.addParticle(atom.element.mass)

# boxVectors = pdb.topology.getPeriodicBoxVectors()
# if boxVectors is not None:
#     system.setDefaultPeriodicBoxVectors(boxVectors[0], boxVectors[1], boxVectors[2])
# print(boxVectors)
system.usesPeriodicBoundaryConditions()

In [ ]:
model = torch.jit.load('/home/enere@usi.ch/CGffap/Models/ChigAllFramesAnglesandDihFixTmux25.pt')

# force =  torch.jit.script(model)

# force.save('ChigAllFramesDatasetResilient.pt')
model.state_dict()

In [ ]:
# list(dataset['bead_types'])

In [ ]:
# list(model.state_dict()['dispertion_const'].cpu().numpy())


In [ ]:
# torch.cuda.set_device(1)
# print(torch.cuda.current_device())

In [ ]:
# SUCCESSFUL TRIAL WITHOUT FORCES AND WITHOUT BONDS (EMPTY SYSTEM WITH JUST THE PARTICLES)

# pdb_file = '/home/enere@usi.ch/FMartIP/ChigCG.pdb'
# pdb = PDBFile(pdb_file) # OpenMM loader
# pdb.topology = Topology()
# pdb.topology.addAtom(dataset['bead_names'][0], "bead", "TRY")

# gro = GromacsGroFile('/home/enere@usi.ch/FMartIP/ChigCG.gro')
# top = GromacsTopFile('/home/enere@usi.ch/FMartIP/ChigCGtopol.top')
# top.setPeriodicBoxVectors(gro.getPeriodicBoxVectors())

# modeller = Modeller(top,gro.positions)

# Save the NNP to a file and load it with OpenMM-Torch
# torch.jit.script(potential).save('model.pt')
# force = TorchForce('model.pt', {'useCUDAGraphs': 'true'})
# force = TorchForce('modeltrain.pt')

model_name = 'ChigAllFramesAnglesandDihFixTmux25.pt'
force = TorchForce('/home/enere@usi.ch/CGffap/Models/' + model_name) #'/home/enere@usi.ch/CGffap/bestmodelchigtestboard.pt'
# #

# I would need to still create the empty system
# forcefield = ForceField('amber14-all.xml')
# system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=None)
# integrator = LangevinIntegrator(300*kelvin, 1/picoseconds , 0.005 * picoseconds)

integrator = VerletIntegrator( 0.010*picoseconds) # NoseHooverIntegrator(300*kelvin, 1/picosecond, 0.010*picoseconds) # #
#
#NoseHooverIntegrator(300*kelvin, 1/picosecond, 0.010*picoseconds)


while system.getNumForces() > 0:
    system.removeForce(0)
    
# The system should not contain any additional force and constrains
assert system.getNumConstraints() == 0
assert system.getNumForces() == 0

# Add the NNP to the system
system.addForce(force)

# This line combines the molecular topology, system, and integrator to begin a new simulation. It creates a Simulation object and assigns it to a variable called simulation. 
# A Simulation object manages all the processes involved in running a simulation, such as advancing time and writing output.
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.getPositions())

# Performs a local energy minimization. It is usually a good idea to do this at the start of a simulation, since the coordinates in the PDB file might produce very large forces.
# simulation.minimizeEnergy()
print("starting Sim")

simulation.reporters.append(PDBReporter( 'Sims/' + model_name + 'output.pdb', 10))
simulation.reporters.append(StateDataReporter('Sims/' + model_name + 'output.dat', 10, step=True, potentialEnergy=True, kineticEnergy=True, temperature=True, time=True, totalEnergy=True))
simulation.reporters.append(ForceReporter('outputforces.txt', 10))

#This line adds another reporter to print out some basic information every 1000 time steps
simulation.step(10000)
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
f = np.array([[a.x,a.y,a.z]for a in state.getForces()])
p = np.array([[a.x,a.y,a.z]for a in state.getPositions()])
# print(state.getForces(), state.getPositions())

In [ ]:
len(pdb.getPositions())

In [ ]:
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)
np.array([[a.x,a.y,a.z]for a in state.getForces()])

In [ ]:
poses = torch.Tensor(p).to('cuda')

In [ ]:
# model = torch.jit.load('/home/enere@usi.ch/CGffap/bestmodelchigtestboard.pt')
# model_forces = model(poses)

In [ ]:
# forcemodel = tm.ForceMapper(model, dataset=dataset, pdb_path= '/home/enere@usi.ch/CGffap/pro.pdb')

# forcemodel(poses).detach().cpu().numpy() == np.array([[a.x,a.y,a.z]for a in state.getForces()])

In [ ]:
output_file = os.path.join(dataset_dir, "enere@usi.ch/CGffap/dataset.notVoid.npz")  #fmartip/ff-naive/DatasetsA2A/dataset.A2a.hydrogen.pose0.npz
dataset = dict(np.load(output_file))

In [ ]:
np.linalg.norm(dataset['bead_forces'],axis=0).max()

In [ ]:
dataset_void = dict(np.load('/home/enere@usi.ch/CGffap/dataset.VoidNoPBC.npz'))
np.linalg.norm(dataset_void['bead_forces'],axis=0).max()

In [ ]:
simForces = np.linalg.norm(np.array(simulation.reporters[2].getForces()),axis=-1)
beadForces = np.linalg.norm(dataset['bead_forces'][:len(simForces)],axis=-1)
beadForces_void = np.linalg.norm(dataset_void['bead_forces'][:len(simForces)],axis=-1)

In [ ]:
# Plot losses
import matplotlib.pyplot as plt
selected_bead_index = 0
# Plotting dataset 1
plt.figure(figsize=(10, 5))
plt.plot(simForces[:,selected_bead_index], label='simForces')


# Plotting dataset 2
plt.plot(beadForces[:,selected_bead_index], linestyle='--', label='beadForces')

# Plotting dataset 3
# plt.plot(beadForces_void[:,selected_bead_index], label='beadForces void')

# Adding labels and title
plt.xlabel('Index')
plt.ylabel('Force')
plt.title('Comparison of Forces for Dataset 1 and Dataset 2')

# Adding legend
plt.legend()

# Displaying the plot
plt.show()

In [ ]:
state = simulation.context.getState(getPositions=True, getEnergy=True, getForces=True)

state.getForces()